In [1]:
# pip install datasets evaluate transformers huggingface_hub -q

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from datasets import DatasetDict, Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
from transformers import DataCollatorWithPadding

## Load Data

In [4]:
dataset_dict = load_dataset("shawhin/phishing-site-classification")

In [5]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 2100
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
})

## Load Pre-trained Model

In [ ]:
import torch

# define pre-trained model path
model_path = "google-bert/bert-base-uncased"

# load model tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# load model with binary classification head
id2label = {0: "not-phishing", 1: "phishing"}
label2id = {"not-phishing": 0, "phishing": 1}
model = AutoModelForSequenceClassification.from_pretrained(
    model_path, num_labels=2, id2label=id2label, label2id=label2id
)

## Set Trainable Parameters (Transfer Learning)

In [ ]:
# freeze all base model parameters
for name, param in model.base_model.named_parameters():
  param.requires_grad = False

# unfreeze base model pooling layers
for name, param in model.base_model.named_parameters():
  if "pooler" in name:
    param.requires_grad = True

## Data Pre-processing

In [ ]:
# define text preprocessing
def preprocess_function(item):
  return tokenizer(item["text"], truncation=True)

# preprocess all data
tokenized_data = dataset_dict.map(preprocess_function, batched=True)


In [ ]:
tokenized_data

In [ ]:
# create data collator to make smaller texts pad to highest text in the batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Define Evaluation Metric

In [ ]:
# load metrics
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    # get predictions (-1 to 1)
    predictions, labels = eval_pred
    
    # apply softmax to get probabilities (-1 to 1 --> 0 to 1)
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)
    # use probabilities of the positive class for ROC AUC
    positive_class_probs = probabilities[:, 1]
    # compute auc
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'], 3)
    
    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)
    # compute accuracy
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'], 3)
    
    return {"Accuracy": acc, "AUC": auc}

## Training Parameters

In [ ]:
# hyperparameters
lr = 2e-4
batch_size = 8
num_epochs = 10

training_args = TrainingArguments(
    output_dir="bert-phishing-classifier_teacher",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
)

## Fine-tune Model

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

## Validate the model

In [ ]:
# apply model to validation dataset
predictions = trainer.predict(tokenized_data["validation"])

# Extract the logits and labels from the predictions object
logits = predictions.predictions
labels = predictions.label_ids

# Use your compute_metrics function
metrics = compute_metrics((logits, labels))
print(metrics)